In [1]:
# imports
import pdal
import json
import os
import time
from math import sqrt
from spatial_lib import run_pipe_with_time



input_file = "output\\processed\\2023-12-22\\lidar_combined.laz"
# input_file = "output\\processed\\2025-03-31\\lidar_combined.laz"
# input_file = "output\\output_small.laz"

In [3]:
# check point spacing
pipeline_json = {
    "pipeline": [
        input_file,
        {
            "type": "filters.stats",
            "dimensions": "X,Y"
        }
    ]
}

p = pdal.Pipeline(json.dumps(pipeline_json))
p.execute()

metadata = p.metadata
stats = metadata["metadata"]["filters.stats"]["statistic"]
las_bounds = metadata["metadata"]["readers.las"]

# Extract count from one dimension (X or Y, both same)
count = stats[0]["count"]

# Extract bounding box from LAS metadata
minx, maxx = las_bounds["minx"], las_bounds["maxx"]
miny, maxy = las_bounds["miny"], las_bounds["maxy"]
area_m2 = (maxx - minx) * (maxy - miny)

# Estimate points per square metre
points_per_m2 = count / area_m2
avg_spacing_m = sqrt(1 / points_per_m2)
avg_spacing_m_rounded = round(avg_spacing_m, 2)

print(f"Input file: {input_file}")
print(f"Point count: {count:,}")
print(f"Area: {area_m2:.2f} m²")
print(f"Avg point spacing: {avg_spacing_m:.2f} m")

Input file: output\processed\2023-12-22\lidar_combined.laz
Point count: 92,990,426
Area: 12000000.00 m²
Avg point spacing: 0.36 m


| Avg Point Spacing | Suggested Resolution |
| ----------------- | -------------------- |
| < 0.1 m           | 0.1 m                |
| \~0.2–0.5 m       | 0.25–0.5 m           |
| \~1.0 m           | 1.0 m                |
| \~2.0 m           | 2.0 m                |


In [4]:
# generate dsm and dtm rasters
resolution = avg_spacing_m_rounded


pipeline = {
    "pipeline": []
}

pipeline["pipeline"].append({ "type":"readers.las",  "filename":input_file })

# write the dsm raster by taking the maximum value in each pixel
pipeline['pipeline'].append({ "type":"writers.gdal",
      "filename":os.path.join(os.path.dirname(input_file), f"dsm_{resolution}.tif"),
      "resolution":resolution,
      "output_type":"max",
      "bounds": "([295000,298000],[6425000,6429000])"
      }
    )
# keep only ground returns (Class 2)
pipeline['pipeline'].append({                                   
      "type": "filters.range",
      "limits": "Classification[2:2]"
    }
    )
# write the dtm raster by taking the min value in each pixel from the filtered data
pipeline['pipeline'].append({ "type":"writers.gdal",
      "filename":os.path.join(os.path.dirname(input_file), f"dtm_{resolution}.tif"),
      "resolution":resolution,
      "output_type":"min",
      "bounds": "([295000,298000],[6425000,6429000])"
      }
    )

# pipeline['num_threads'] = 8


print(json.dumps(pipeline, indent=2))
run_pipe_with_time(pdal.Pipeline(json.dumps(pipeline)), streaming=True)

{
  "pipeline": [
    {
      "type": "readers.las",
      "filename": "output\\processed\\2023-12-22\\lidar_combined.laz"
    },
    {
      "type": "writers.gdal",
      "filename": "output\\processed\\2023-12-22\\dsm_0.36.tif",
      "resolution": 0.36,
      "output_type": "max",
      "bounds": "([295000,298000],[6425000,6429000])"
    },
    {
      "type": "filters.range",
      "limits": "Classification[2:2]"
    },
    {
      "type": "writers.gdal",
      "filename": "output\\processed\\2023-12-22\\dtm_0.36.tif",
      "resolution": 0.36,
      "output_type": "min",
      "bounds": "([295000,298000],[6425000,6429000])"
    }
  ]
}
Starting PDAL pipeline execution in streaming mode...
Pipeline execution complete. Processed 65704094 points in 366.94 seconds.
Pipeline execution complete. Processed 65704094 points in 366.94 seconds.


In [5]:
import subprocess

# fill the nodata values in the dtm raster
# Use gdal_fillnodata.bat output/dtm_small_fill_20_0_0.2.tif output/dtm_small_fill_20_0_0.2.tif -md 20 -b 1 -of GTiff
dtm = os.path.join(os.path.dirname(input_file), f"dtm_{resolution}.tif")
outfile = os.path.join(os.path.dirname(input_file), f"dtm_filled_{resolution}.tif")

command = f"micromamba run -n geo_env gdal_fillnodata {dtm} {outfile} -md 20 -b 1 -of GTiff"
print(command)
result = subprocess.run(command, shell=True, capture_output=True, text=True)
print(f"Return code: {result.returncode}")
if result.stdout:
    print(f"Output: {result.stdout}")
if result.stderr:
    print(f"Error: {result.stderr}")
# os.system(command)

micromamba run -n geo_env gdal_fillnodata output\processed\2023-12-22\dtm_0.36.tif output\processed\2023-12-22\dtm_filled_0.36.tif -md 20 -b 1 -of GTiff
Return code: 0
Output: 0...10...20...30...40...50...60...70...80...90...100 - done.

Return code: 0
Output: 0...10...20...30...40...50...60...70...80...90...100 - done.



In [6]:
# construct the chm command line and shell it out
dsm = os.path.join(os.path.dirname(input_file), f"dsm_{resolution}.tif")
dtm = os.path.join(os.path.dirname(input_file), f"dtm_filled_{resolution}.tif")
outfile = os.path.join(os.path.dirname(input_file), f"chm_{resolution}.tif")
# command = f"gdal_calc.py -A {dsm} -B {dtm} --outfile={outfile} --calc=\"A-B\" --NoDataValue=-9999 "
# print(dsm)
command = f"micromamba run -n geo_env gdal_calc -A {dsm} -B {dtm} --outfile={outfile} --calc=\"A-B\" --NoDataValue=-9999 "
print(command)
result = subprocess.run(command, shell=True, capture_output=True, text=True)
print(f"Return code: {result.returncode}")
if result.stdout:
    print(f"Output: {result.stdout}")
if result.stderr:
    print(f"Error: {result.stderr}")
# os.system(command)

micromamba run -n geo_env gdal_calc -A output\processed\2023-12-22\dsm_0.36.tif -B output\processed\2023-12-22\dtm_filled_0.36.tif --outfile=output\processed\2023-12-22\chm_0.36.tif --calc="A-B" --NoDataValue=-9999 
Return code: 0
Output: 0...10...20...30...40...50...60...70...80...90...100 - done.

Return code: 0
Output: 0...10...20...30...40...50...60...70...80...90...100 - done.

